<a href="https://colab.research.google.com/github/100jy/Dacon_card/blob/main/%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C_%EC%82%AC%EC%9A%A9%EC%9E%90_%EC%97%B0%EC%B2%B4_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import os
os.chdir('/content/drive/MyDrive/데이콘')

In [39]:
train = pd.read_csv('train.csv', index_col=0)
train = train.fillna('NA')

# 변수확인

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for i in range(train.shape[1]):
  plt.figure()
  try:
    sns.kdeplot(train[:,i])
  except:
    plt.hist(train[:,i])
  plt.title(train.columns[i])

# 전처리

In [36]:
train

index
0        -6.0
1        -5.0
2       -22.0
3       -37.0
4       -26.0
         ... 
26452    -2.0
26453   -47.0
26454   -25.0
26455   -59.0
26456    -9.0
Name: begin_month, Length: 26457, dtype: float64

In [86]:
# 범주형 인코딩
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder

train = pd.read_csv('train.csv', index_col=0)
train = train.fillna('NA')

sub = pd.read_csv('sample_submission.csv', index_col=0)
test  = pd.read_csv('test.csv', index_col=0)
test = test.fillna('NA')

for cate in ['gender', 'car', 'reality', 'income_type',
       'edu_type', 'family_type', 'house_type', 
     'occyp_type',
     'family_size', 'begin_month']:
  inco = LabelEncoder() 
  inco.fit(train[cate])
  train[cate] = inco.transform(train[cate])
  test[cate] = inco.transform(test[cate])


X = train.iloc[:,:-1]
Y = train.iloc[:,[-1]].values

# 모든 변수 minmax
scaler  = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)
test = scaler.transform(test)

# 모델링

In [92]:
from sklearn.metrics import log_loss, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from keras.utils import to_categorical


# StratifiedKfold(3)
# train val subsampling하여 여러 모델 피팅 후 앙상블
fold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
total = 0
re = np.zeros_like(sub.values, dtype=np.float32)

for train_idx, val_idx in fold.split(X, Y):
    train_x, val_x, train_y, val_y =  X[train_idx], X[val_idx],\
                                      Y[train_idx], Y[val_idx]
    # 랜포
    clf = RandomForestClassifier()
    clf.fit(train_x, train_y)
    prob = clf.predict_proba(val_x)
    loss = log_loss(to_categorical(val_y), prob)
    re += clf.predict_proba(test)
    total += loss
    print('val_loss : {:.3f}'.format(loss))

print('mean_loss : {:.3f}'.format(total/3))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


val_loss : 0.895


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


val_loss : 0.893


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


val_loss : 0.920
mean_loss : 0.903


In [93]:
sub.iloc[:,:] = re / 3

In [95]:
sub.to_csv('baseline_submission.csv', index=False)